In [2]:
from spektral import datasets
from spektral.layers import CensNetConv

# Load the citation data.
dataset = datasets.citation.Citation("cora", normalize_x=True, random_splits=False)
cora = dataset.read()[0]

Pre-processing node features
Pre-processing node features


In [3]:
import tensorflow as tf

# Convert citation graph into an undirected graph.
adjacency = cora.a.todense()
adjacency_upper = tf.linalg.band_part(adjacency, 0, -1)
adjacency_lower = tf.linalg.band_part(adjacency, -1, 0)

adjacency_upper_symmetric = adjacency_upper + tf.transpose(adjacency_upper)
adjacency_lower_symmetric = adjacency_lower + tf.transpose(adjacency_lower)
adjacency_undirected = tf.maximum(adjacency_upper_symmetric, adjacency_lower_symmetric)

2022-06-30 16:50:48.515342: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-30 16:50:48.515366: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-30 16:50:48.515381: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (twilight-sparkle): /proc/driver/nvidia/version does not exist
2022-06-30 16:50:48.515574: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
from cotton_flower_mot.pipelines.model_training.graph_utils import compute_pairwise_similarities
from cotton_flower_mot.pipelines.model_training.similarity_utils import cosine_similarity
from spektral.data import Graph
tf.keras.mixed_precision.set_global_policy("float32")

# Compute edge features.
connected_node_indices = tf.where(adjacency_upper)
# Get the corresponding node features for each edge.
left_node_features = tf.gather(cora.x, connected_node_indices[:, 0])
right_node_features = tf.gather(cora.x, connected_node_indices[:, 1])
# Compute cosine similarities for each edge.
cosine_similarities = cosine_similarity(left_node_features, right_node_features)

edge_features = tf.expand_dims(cosine_similarities, -1)
cora = Graph(x=cora.x, a=cora.a, e=edge_features.numpy(), y=cora.y)

In [5]:
from tensorflow.keras import Input, Model, layers
from spektral.layers import CensNetConv

# Build the model.
node_features = Input(shape=(cora.n_node_features,))
edge_features = Input(shape=(cora.n_edge_features,))
node_laplacian = Input(shape=(cora.n_nodes,))
# The undirected graph means that the number of edges is doubled.
edge_laplacian = Input(shape=(cora.n_edges // 2,))
incidence = Input(shape=(cora.n_edges // 2,))

static_features = (node_laplacian, edge_laplacian, incidence)

nodes_2, edges_2 = CensNetConv(64, 64, activation="relu")((node_features, static_features, edge_features))
nodes_2 = layers.Dropout(0.5)(nodes_2)
edges_2 = layers.Dropout(0.5)(edges_2)
nodes_3, _ = CensNetConv(64, 64, activation="relu")((nodes_2, static_features, edges_2))
nodes_3 = layers.Dropout(0.5)(nodes_3)
# Apply the classification.
node_class = layers.Dense(cora.n_labels, activation="softmax")(nodes_3)

model = Model(inputs=[node_features, edge_features, node_laplacian, edge_laplacian, incidence], outputs=[node_class])

In [6]:
from spektral.data.loaders import SingleLoader
import numpy as np

# Prepare for training.
model.compile(optimizer=tf.keras.optimizers.Adam(0.01),
              loss=tf.keras.losses.CategoricalCrossentropy(reduction="sum"),
              weighted_metrics=["acc"])

# We convert the binary masks to sample weights so that we can compute the
# average loss over the nodes (following original implementation by
# Kipf & Welling)
def mask_to_weights(mask):
    return mask.astype(np.float32) / np.count_nonzero(mask)


weights_tr, weights_va, weights_te = (
    mask_to_weights(mask)
    for mask in (dataset.mask_tr, dataset.mask_va, dataset.mask_te)
)

node_laplacian, edge_laplacian, incidence = CensNetConv.preprocess(cora.a.todense())

inputs_and_targets = ((cora.x, cora.e, node_laplacian, edge_laplacian, incidence), cora.y)
training_dataset = tf.data.Dataset.from_tensors(inputs_and_targets + (weights_tr,))
testing_dataset = tf.data.Dataset.from_tensors(inputs_and_targets + (weights_te,))

In [7]:
tf.keras.mixed_precision.set_global_policy("float32")

# Train the model.
model.fit(training_dataset, validation_data=testing_dataset, epochs=50)

Epoch 1/50
2022-06-30 16:51:20,539 - tensorflow - WARNING - Gradients do not exist for variables ['cens_net_conv_1/edge_kernel:0', 'cens_net_conv_1/node_weights:0', 'cens_net_conv_1/edge_bias:0'] when minimizing the loss. If you're using `model.compile()`, did you forget to provide a `loss`argument?
2022-06-30 16:51:20,698 - tensorflow - WARNING - Gradients do not exist for variables ['cens_net_conv_1/edge_kernel:0', 'cens_net_conv_1/node_weights:0', 'cens_net_conv_1/edge_bias:0'] when minimizing the loss. If you're using `model.compile()`, did you forget to provide a `loss`argument?
1/1 [==============================] - 3s 3s/step - loss: 1.9459 - acc: 0.1071 - val_loss: 1.9433 - val_acc: 0.3190
Epoch 2/50
1/1 [==============================] - 2s 2s/step - loss: 1.9469 - acc: 0.1286 - val_loss: 1.9451 - val_acc: 0.1030
Epoch 3/50
1/1 [==============================] - 2s 2s/step - loss: 1.9450 - acc: 0.1857 - val_loss: 1.9461 - val_acc: 0.0910
Epoch 4/50
1/1 [=======================